## 자동평가 실습
### 활용 데이터: hw_sample.json

### 실습 개요
이 실습에서는 hw_sample.json 파일에 포함된 sentence1, sentence2 두 개의 텍스트에 대해 자동평가 하는 방법을 실습합니다.


### 실습 절차
1. 라이브러리 설치 및 임포트: 필요한 라이브러리를 설치하고 임포트합니다.
2. CSV 파일 로드: pandas를 사용하여 주어진 CSV 파일을 읽고 DataFrame으로 변환합니다.
3. 주어진 두 텍스트에 대해 각각 METEOR, LASER, chrF, BERTScore를 계산합니다.
4. 결과 저장 및 비교: 두 방법으로 계산된 점수를 각각 새로운 열에 저장하고, 결과를 비교 분석합니다.
5. 최종 결과 저장: 계산된 점수가 포함된 DataFrame을 metrics_results.csv 파일로 저장합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets sacrebleu bert_score

In [ ]:
!pip install laserembeddings[ko]

In [4]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [5]:
from laserembeddings import Laser
import numpy as np
from scipy.spatial.distance import cosine
import pandas as pd
from datasets import load_metric
from laserembeddings import Laser
import json

In [6]:
meteor = load_metric('meteor')
chrf = load_metric('chrf')
bertscore = load_metric("bertscore")

<ipython-input-6-2f3f7ea6e34f>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  meteor = load_metric('meteor')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for meteor contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/meteor/meteor.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for chrf contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/chrf/chrf.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for bertscore contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/bertscore/bertscore.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [7]:
with open("/content/drive/MyDrive/NLP 실습/1 18/hw_sample.json") as f:
  data = json.load(f)

In [8]:
data.keys()

dict_keys(['data'])

In [9]:
len(data['data'])

1

In [13]:
data['data'][0].keys()

dict_keys(['paragraph_info', 'sentences'])

In [14]:
len(data['data'][0]['sentences'])

30

In [15]:
df = pd.DataFrame(data['data'][0]['sentences'])

In [17]:
df['meteor'] = df.apply(lambda x: meteor.compute(predictions=[x['sentence1']], references=[x['sentence2']])['meteor'], axis=1)

In [20]:
df['chrf'] = df.apply(lambda x: chrf.compute(predictions=[x['sentence1']], references=[[x['sentence2']]])['score'], axis=1)

In [21]:
df['bertscore_model'] = df.apply(lambda x: bertscore.compute(predictions=[x['sentence1']], references=[x['sentence2']], lang="ko"), axis=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

In [22]:
df.head(2)

,sentence1,sentence2,meteor,chrf,bertscore_model
0,1990년대와 2000년대는 마케팅 분야에서 브랜드관리가 매우 부상했다.,애경과 현대카드에서 마케팅팀장을 역임한 저자는 서울시 도시마케팅 전담기구로 출범한 ...,0.025381,4.876903,"{'precision': [0.7124655842781067], 'recall': ..."
1,이렇게 커피 시장이 커지고 경쟁이 치열해지다 보니 커피 업체의 마케팅도 치열하다.,현대경영에서 마케팅의 중요성을 강조하는 것도 이미 진부한 일이다.,0.053763,8.374007,"{'precision': [0.7190718054771423], 'recall': ..."


In [23]:
import numpy as np
from scipy.spatial.distance import cosine

# LASER 모델 초기화
mylaser = Laser()

In [24]:
# 문장 임베딩 생성
df['sentence1_laser_embeddings'] = df.apply(lambda x: mylaser.embed_sentences([x['sentence1']], lang='ko'), axis=1)
df['sentence2_laser_embeddings'] = df.apply(lambda x: mylaser.embed_sentences([x['sentence2']], lang='ko'), axis=1)

In [25]:
def cosine_similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2.T).squeeze() / n1 / n2

In [26]:
df['laser_cosine_similarity2'] = df.apply(lambda x: cosine_similarity(x['sentence1_laser_embeddings'], x['sentence2_laser_embeddings']), axis=1)

In [27]:
df.head(2)

,sentence1,sentence2,meteor,chrf,bertscore_model,sentence1_laser_embeddings,sentence2_laser_embeddings,laser_cosine_similarity2
0,1990년대와 2000년대는 마케팅 분야에서 브랜드관리가 매우 부상했다.,애경과 현대카드에서 마케팅팀장을 역임한 저자는 서울시 도시마케팅 전담기구로 출범한 ...,0.025381,4.876903,"{'precision': [0.7124655842781067], 'recall': ...","[[0.005228753, 0.008393525, -4.69585e-05, 0.04...","[[0.011531885, 0.0007702296, 0.002648675, 0.02...",0.625797
1,이렇게 커피 시장이 커지고 경쟁이 치열해지다 보니 커피 업체의 마케팅도 치열하다.,현대경영에서 마케팅의 중요성을 강조하는 것도 이미 진부한 일이다.,0.053763,8.374007,"{'precision': [0.7190718054771423], 'recall': ...","[[0.0064634727, 0.018033417, 0.00023138202, 0....","[[0.0038576797, 0.00026826354, -0.00022856104,...",0.659561


In [28]:
df.to_csv('metrics_results.csv', index=False)